In [13]:
# # !pip install torch
# ! pip install fastai

In [14]:
from fastai import *
from fastai.collab import *

In [16]:
# from fastai.tabular.all import *
# from fastai.collab import *

In [18]:
path = untar_data(URLs.ML_SAMPLE)

In [20]:
ratings = pd.read_csv(path/'ratings.csv')
ratings.head()

,userId,movieId,rating,timestamp
0,73,1097,4.0,1255504951
1,561,924,3.5,1172695223
2,157,260,3.5,1291598691
3,358,1210,5.0,957481884
4,130,316,2.0,1138999234


In [21]:
data = CollabDataBunch.from_df(ratings, seed=42)

In [22]:
y_range = [0,5.2]
learn = collab_learner(data, n_factors=50, y_range=y_range)

In [23]:
y_range = [0,5.2]
learn = collab_learner(data, n_factors=50, y_range=y_range)

In [24]:
learn.fit_one_cycle(3, 5e-3)

epoch,train_loss,valid_loss,time
0,1.878807,1.058160,00:01
1,0.897821,0.681033,00:01
2,0.674148,0.674133,00:01


In [41]:
rating_movie=pd.read_csv('/content/rating_movie_mergedata_select.csv')

In [42]:
rating_movie.head()

,Unnamed: 0,userId,movieId,rating,timestamp,title
0,0,1,296,5.0,1147880044,Pulp Fiction (1994)
1,1,3,296,5.0,1439474476,Pulp Fiction (1994)
2,2,4,296,4.0,1573938898,Pulp Fiction (1994)
3,3,5,296,4.0,830786155,Pulp Fiction (1994)
4,4,7,296,4.0,835444730,Pulp Fiction (1994)


In [43]:
rating_movie_selec_features=rating_movie[['userId','title','rating']]

In [44]:
rating_movie_selec_features.head()

,userId,title,rating
0,1,Pulp Fiction (1994),5.0
1,3,Pulp Fiction (1994),5.0
2,4,Pulp Fiction (1994),4.0
3,5,Pulp Fiction (1994),4.0
4,7,Pulp Fiction (1994),4.0


In [45]:
data = CollabDataBunch.from_df(rating_movie_selec_features, seed=42, valid_pct=0.1, item_name='title')
data.show_batch()

userId,title,target
92439,Pulp Fiction (1994),4.5
28311,Pulp Fiction (1994),3.0
162440,Pulp Fiction (1994),2.0
142470,Pulp Fiction (1994),4.5
125882,Pulp Fiction (1994),5.0


In [46]:
learn = collab_learner(data, n_factors=40, y_range=y_range, wd=1e-1)
learn.fit_one_cycle(6, 5e-3)

epoch,train_loss,valid_loss,time
0,0.955376,0.944203,00:16
1,0.801970,0.940549,00:16
2,0.481537,0.958255,00:16
3,0.277413,0.922589,00:17
4,0.136094,0.932806,00:16
5,0.064827,0.945594,00:17


In [48]:
learn.save('movie-reviews')

In [49]:
g = rating_movie_selec_features.groupby('title')['rating'].count()
top_movies = g.sort_values(ascending=False).index.values[:10000]
top_movies[:10]
movie_bias = learn.bias(top_movies, is_item=True)


In [50]:
top_movies

array(['Pulp Fiction (1994)', 'Three Colors: Red (Trois couleurs: Rouge) (1994)',
       'Three Colors: Blue (Trois couleurs: Bleu) (1993)', "Singin' in the Rain (1952)", 'Underground (1995)'],
      dtype=object)

In [51]:
ratings = [(b, i) for i,b in zip(top_movies,movie_bias)]
sorted(ratings)[:15]

[(tensor(0.7648), 'Underground (1995)'),
 (tensor(0.8581), 'Three Colors: Blue (Trois couleurs: Bleu) (1993)'),
 (tensor(0.9286), 'Three Colors: Red (Trois couleurs: Rouge) (1994)'),
 (tensor(1.0149), "Singin' in the Rain (1952)"),
 (tensor(1.1615), 'Pulp Fiction (1994)')]